We used Rose chart to reveal seasonal spending patterns for categories like Utilities, groceries and Vacation. The inspiration for using the rose chart came from Florence Nightingale’s “rose chart” diagrams, which were taught in class as a pioneering example of how visualizations can effectively communicate complex data.

The first step is installing the Dash framework, including its core components (dash, dash-core-components, dash-html-components, and dash-table).

In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


## Developing a Rose Chart

1. Data Loading and Preprocessing - uses pd.read_excel() to load the specified sheet (Sheet1) from the Excel file into a Pandas DataFrame (sheet1_data), which serves as the dataset for our visualization.

2. Extracting column names related to the categories (Groceries, Utilities, Vacation) from the dataset.

3. mean(axis=0).values - For Computing the average spending for each category (Groceries, Utilities, Vacation) across months by taking the mean of corresponding columns and Converts the result into an array of numerical values for easy plotting.

4. Dash Application Setup - Defining the layout of the app.
dcc.Graph: The core component that renders the wind rose chart.
id='interactive-wind-rose': Assigns an ID to the graph for callback updates.
config={'displayModeBar': False}: Disables the mode bar (tool options) to declutter the interface.
style: Specifies the size of the chart.

5. Interactive Callback Function - Defines the interaction logic for updating the chart dynamically.
Output: Updates the figure property of the graph with ID 'interactive-wind-rose'.
Input: Tracks user interactions, such as hovering over the chart (hoverData).

6. Visualization Creation - To build the rose chart.
fig = go.Figure(): Initializes a Plotly figure

7. Adding Data - Adds a Barpolar trace (a polar bar chart)
r=groceries_avg: Radial values (lengths of the bars) represent average spending on groceries.
theta=months: Angular positions correspond to months.
name: Legend name for this trace.
marker_color: Defines the color of the bars.
opacity: Adjusts the visibility of the bars dynamically based on user hover.
hovertemplate: Specifies the hover text template for better interactivity.
Similar logic performed for utilities and vacation category too.

8. Chart Customization - Customizes the layout and appearance of the rose chart.
polar: Defines the polar coordinate system.
Angular axis: Set to a clockwise direction with 90° rotation.
Radial axis: Dynamically adjusts the range based on the highest spending.
Colors: Dark mode (paper_bgcolor and plot_bgcolor) for aesthetics.
Legend: Positioned at the bottom for clarity.

9. Running the App - Launches the Dash app.
debug=True: Enables live reloading and detailed error messages during development.


In [2]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

# Load your dataset
file_path = '/content/Money_MindScape Jot form - almost final.xlsx'
sheet1_data = pd.read_excel(file_path, sheet_name='Sheet1')

# Extract relevant columns
groceries_columns = [col for col in sheet1_data.columns if 'Groceries' in col]
utilities_columns = [col for col in sheet1_data.columns if 'Utilities' in col]
vacation_columns = [col for col in sheet1_data.columns if 'Vacation' in col]

# Compute average values for the categories
groceries_avg = sheet1_data[groceries_columns].mean(axis=0).values
utilities_avg = sheet1_data[utilities_columns].mean(axis=0).values
vacation_avg = sheet1_data[vacation_columns].mean(axis=0).values

# Define months for the angular axis
months = [col.split('_')[1] for col in groceries_columns]

# Create the app
app = Dash(__name__)

# Layout of the Dash app
app.layout = html.Div([
    dcc.Graph(
        id='interactive-wind-rose',
        config={'displayModeBar': False},
        style={'height': '800px', 'width': '100%'}  # Adjust height and width here
    )
])

# Callback to dynamically update the chart
@app.callback(
    Output('interactive-wind-rose', 'figure'),
    Input('interactive-wind-rose', 'hoverData')
)
def update_chart(hover_data):
    # Determine which trace is being hovered over
    hovered_trace = None
    if hover_data and 'points' in hover_data:
        hovered_trace = hover_data['points'][0]['curveNumber']

    # Initialize the figure
    fig = go.Figure()

    # Add Groceries data
    fig.add_trace(go.Barpolar(
        r=groceries_avg,
        theta=months,
        name='🍎 Groceries',
        marker_color='rgba(255, 165, 0, 0.7)',  # Semi-transparent orange
        opacity=1.0 if hovered_trace == 0 else 0.3,
        hoverinfo='text',
        hovertemplate="<b>%{theta}</b><br>Groceries: %{r}<extra></extra>"
    ))

    # Add Utilities data
    fig.add_trace(go.Barpolar(
        r=utilities_avg,
        theta=months,
        name='💡 Utilities',
        marker_color='rgba(0, 123, 255, 0.7)',  # Semi-transparent blue
        opacity=1.0 if hovered_trace == 1 else 0.3,
        hoverinfo='text',
        hovertemplate="<b>%{theta}</b><br>Utilities: %{r}<extra></extra>"
    ))

    # Add Vacation data (instead of Subscriptions)
    fig.add_trace(go.Barpolar(
        r=vacation_avg,
        theta=months,
        name='🏖 Vacation',
        marker_color='rgba(0, 255, 0, 0.7)',  # Semi-transparent green
        opacity=1.0 if hovered_trace == 2 else 0.3,
        hoverinfo='text',
        hovertemplate="<b>%{theta}</b><br>Vacation: %{r}<extra></extra>"
    ))


    # Update layout for customization (Dark mode, radial axis, etc.)
    fig.update_layout(
        title='Interactive Wind Rose Chart',
        polar=dict(
            angularaxis=dict(direction='clockwise', rotation=90, tickfont=dict(color='white')),
            radialaxis=dict(visible=True, range=[0, max(max(groceries_avg), max(utilities_avg), max(vacation_avg))], showticklabels=False, gridcolor='rgba(255,255,255,0.1)'),
        ),
        paper_bgcolor="rgb(30,30,30)",
        plot_bgcolor="rgb(30,30,30)",
        font=dict(color="white"),
        hovermode='closest',
        showlegend=True,
        legend=dict(
            title="Category",
            font=dict(size=12),
            orientation="h",  # Horizontal legend
            x=0.5,
            xanchor="center",
            y=-0.1,
        ),
        transition=dict(
            duration=500,
            easing='cubic-in-out'
        ),
        width=1000,  # Increase width for larger graph
        height=600   # Increase height for larger graph
    )

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>

When users hover over a specific category, such as Groceries, Utilities, or Vacation, the chart responds by emphasizing the selected category while dimming the others. This hover-based interaction allows users to focus on individual data points without losing sight of the overall context.